In [3]:
from keras.models import Sequential
from keras import layers
import numpy as np
from six.moves import range


# Parameters Config

In [4]:
class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'


In [11]:
TRAINING_SIZE = 50000
DIGITS = 3

REVERSE = True

MAXLEN = DIGITS + 1 + DIGITS
chars = '0123456789- '

RNN = layers.LSTM
HIDDEN_SIZE = 128
BATCH_SIZE = 128
LAYERS = 1


# Encoder & Decode

In [12]:
class CharacterTable(object):
    def __init__(self, chars):
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))
    
    def encode(self, C, num_rows):
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x
    
    def decode(self, x, calc_argmax=True):
        if calc_argmax:
            x = x.argmax(axis=-1)
        return "".join(self.indices_char[i] for i in x)


In [13]:
ctable = CharacterTable(chars)


# Data Generation, only subtractor

In [14]:
questions = []
expected = []
seen = set()

print('Generating data...')
while len(questions) < TRAINING_SIZE:
    f = lambda: int(''.join(np.random.choice(list('0123456789'))
                            for i in range(np.random.randint(1, DIGITS + 1))))
    a, b = f(), f()
    
    key = tuple(sorted((a, b)))
    if key in seen:
        continue
    seen.add(key)
    # Pad the data with spaces such that it is always MAXLEN.
    q = '{}-{}'.format(a, b)
    query = q + ' ' * (MAXLEN - len(q))
    
    ans = str(a - b)
    # Answers can be of maximum size DIGITS + 1.
    ans += ' ' * (DIGITS + 1 - len(ans))

    questions.append(query)
    expected.append(ans)
print('Total addition questions:', len(questions))



Generating data...
Total addition questions: 50000


In [15]:
print(questions[:5], expected[:5])


['46-70  ', '5-105  ', '8-2    ', '12-4   ', '2-20   '] ['-24 ', '-100', '6   ', '8   ', '-18 ']


# Data Processing

In [16]:
print('Vectorization...')
x = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
y = np.zeros((len(expected), DIGITS + 1, len(chars)), dtype=np.bool)
for i, sentence in enumerate(questions):
    x[i] = ctable.encode(sentence, MAXLEN)
for i, sentence in enumerate(expected):
    y[i] = ctable.encode(sentence, DIGITS + 1)
print('Done...')

Vectorization...


# Shuffle & Split train, test, validation data

In [17]:
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

# train_test_split
train_x = x[:30000]
train_y = y[:30000]
test_x = x[30000:]
test_y = y[30000:]

split_at = len(train_x) - len(train_x) // 10
(x_train, x_val) = train_x[:split_at], train_x[split_at:]
(y_train, y_val) = train_y[:split_at], train_y[split_at:]

print('Training Data:')
print(x_train.shape)
print(y_train.shape)

print('Validation Data:')
print(x_val.shape)
print(y_val.shape)

print('Testing Data:')
print(test_x.shape)
print(test_y.shape)



Training Data:
(27000, 7, 12)
(27000, 4, 12)
Validation Data:
(3000, 7, 12)
(3000, 4, 12)
Testing Data:
(20000, 7, 12)
(20000, 4, 12)


In [14]:
#print("input: ", x_train[:3], '\n\n', "label: ", y_train[:3])


# Build Model

In [18]:
print('Build model...')

############################################
##### Build your own model here ############

model = Sequential()

# "Encode" the input sequence using an RNN, input_shape=(None, num_feature).
model.add(RNN(HIDDEN_SIZE, input_shape=(MAXLEN, len(chars))))

# the decoder RNN's input
model.add(layers.RepeatVector(DIGITS + 1))

# The decoder RNN  with a single layer.
for _ in range(LAYERS):
    # all the outputs in the form of (num_samples, timesteps, output_dim). 
    # the first dimension to be the timesteps.
    model.add(RNN(HIDDEN_SIZE, return_sequences=True))


model.add(layers.TimeDistributed(layers.Dense(len(chars), activation='softmax')))

############################################
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()


Build model...
Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               72192     
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 4, 128)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 4, 128)            131584    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 4, 12)             1548      
Total params: 205,324
Trainable params: 205,324
Non-trainable params: 0
_________________________________________________________________


# Training

In [19]:
for iteration in range(100):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x_train, y_train,
              batch_size=BATCH_SIZE,
              epochs=1,
              validation_data=(x_val, y_val))
    # visualizing validation set prediction errors.
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)



--------------------------------------------------
Iteration 0
Train on 27000 samples, validate on 3000 samples
Epoch 1/1
27000/27000 [==============================] - 5s 180us/step - loss: 2.0397 - acc: 0.2963 - val_loss: 1.7557 - val_acc: 0.3543
Q  586-04 T -645 ☒ -720
Q 043-421 T -216 ☒ -23 
Q    5-68 T 81   ☒ 11  
Q  348-88 T -755 ☒ -788
Q  716-93 T -578 ☒ -722
Q   1-868 T 867  ☒ 67  
Q  338-89 T -735 ☒ -788
Q   49-39 T -1   ☒ -2  
Q  038-09 T -740 ☒ -228
Q 472-252 T -22  ☑ -22 

--------------------------------------------------
Iteration 1
Train on 27000 samples, validate on 3000 samples
Epoch 1/1
27000/27000 [==============================] - 4s 163us/step - loss: 1.7010 - acc: 0.3745 - val_loss: 1.7034 - val_acc: 0.3868
Q  89-985 T 491  ☒ 69  
Q 221-141 T 19   ☒ 11  
Q  935-77 T -462 ☒ -30 
Q   732-0 T -237 ☒ -422
Q 539-451 T -781 ☒ -30 
Q 652-967 T 513  ☒ 694 
Q   1-981 T 188  ☒ 89  
Q  83-578 T 837  ☒ 797 
Q  765-92 T -538 ☒ -622
Q  321-17 T -52  ☒ -1  

-------------------

27000/27000 [==============================] - 8s 281us/step - loss: 1.0572 - acc: 0.6129 - val_loss: 1.0786 - val_acc: 0.6005
Q   9-995 T 590  ☒ 599 
Q 787-287 T -5   ☒ -9  
Q 443-148 T 497  ☒ 480 
Q 383-151 T -232 ☒ -235
Q  048-32 T -817 ☒ -803
Q 772-974 T 202  ☒ 122 
Q 814-902 T -209 ☒ -219
Q   686-0 T -686 ☑ -686
Q   7-516 T 608  ☑ 608 
Q   2-853 T 356  ☒ 343 

--------------------------------------------------
Iteration 15
Train on 27000 samples, validate on 3000 samples
Epoch 1/1
27000/27000 [==============================] - 8s 301us/step - loss: 1.0243 - acc: 0.6256 - val_loss: 1.0512 - val_acc: 0.6115
Q  59-689 T 891  ☒ 807 
Q  042-94 T -191 ☒ -175
Q   558-5 T -850 ☒ -849
Q 548-847 T -97  ☒ -10 
Q  92-949 T 920  ☒ 935 
Q  01-885 T 578  ☒ 572 
Q 918-448 T 25   ☒ 1   
Q  498-86 T -826 ☒ -829
Q  98-159 T 862  ☒ 855 
Q  494-25 T -442 ☑ -442

--------------------------------------------------
Iteration 16
Train on 27000 samples, validate on 3000 samples
Epoch 1/1
27000/27000 [=====

27000/27000 [==============================] - 8s 303us/step - loss: 0.2248 - acc: 0.9377 - val_loss: 0.2694 - val_acc: 0.9128
Q  01-671 T 166  ☒ 165 
Q  017-19 T -619 ☒ -629
Q   82-34 T 15   ☑ 15  
Q 124-328 T 402  ☒ 392 
Q  681-03 T -156 ☑ -156
Q  42-661 T 142  ☑ 142 
Q  05-649 T 896  ☑ 896 
Q   85-97 T 21   ☑ 21  
Q   206-7 T -595 ☑ -595
Q   7-189 T 974  ☑ 974 

--------------------------------------------------
Iteration 43
Train on 27000 samples, validate on 3000 samples
Epoch 1/1
27000/27000 [==============================] - 8s 303us/step - loss: 0.2224 - acc: 0.9364 - val_loss: 0.2639 - val_acc: 0.9135
Q 791-728 T 630  ☒ 620 
Q 659-955 T -397 ☒ -497
Q 052-458 T 604  ☒ 503 
Q  715-32 T -494 ☑ -494
Q   57-02 T -55  ☑ -55 
Q 094-071 T -320 ☑ -320
Q   669-8 T -958 ☑ -958
Q   57-42 T -51  ☑ -51 
Q   18-38 T 2    ☒ 1   
Q 238-818 T -14  ☑ -14 

--------------------------------------------------
Iteration 44
Train on 27000 samples, validate on 3000 samples
Epoch 1/1
27000/27000 [=====

27000/27000 [==============================] - 8s 285us/step - loss: 0.0558 - acc: 0.9881 - val_loss: 0.1239 - val_acc: 0.9547
Q 481-528 T 641  ☑ 641 
Q  32-864 T 445  ☑ 445 
Q 113-544 T 134  ☑ 134 
Q   57-02 T -55  ☑ -55 
Q  29-467 T 672  ☑ 672 
Q 293-618 T 424  ☑ 424 
Q  695-36 T -533 ☑ -533
Q 409-606 T -298 ☑ -298
Q  957-38 T -676 ☑ -676
Q  82-951 T 131  ☑ 131 

--------------------------------------------------
Iteration 71
Train on 27000 samples, validate on 3000 samples
Epoch 1/1
27000/27000 [==============================] - 8s 282us/step - loss: 0.0668 - acc: 0.9833 - val_loss: 0.1486 - val_acc: 0.9453
Q  249-93 T -903 ☑ -903
Q  43-207 T 668  ☑ 668 
Q  23-456 T 622  ☑ 622 
Q  548-55 T -790 ☑ -790
Q 474-655 T 82   ☑ 82  
Q 617-732 T -479 ☒ -489
Q   19-31 T -78  ☑ -78 
Q  175-96 T -502 ☑ -502
Q   0-902 T 209  ☑ 209 
Q  01-638 T 826  ☑ 826 

--------------------------------------------------
Iteration 72
Train on 27000 samples, validate on 3000 samples
Epoch 1/1
27000/27000 [=====

27000/27000 [==============================] - 7s 274us/step - loss: 0.0217 - acc: 0.9973 - val_loss: 0.0838 - val_acc: 0.9709
Q  17-874 T 407  ☑ 407 
Q  808-88 T -720 ☑ -720
Q  03-573 T 345  ☑ 345 
Q 128-706 T -214 ☑ -214
Q  82-071 T 142  ☑ 142 
Q  79-801 T 11   ☑ 11  
Q  25-972 T 227  ☑ 227 
Q  081-26 T -118 ☑ -118
Q   6-268 T 856  ☑ 856 
Q   88-73 T -51  ☑ -51 

--------------------------------------------------
Iteration 99
Train on 27000 samples, validate on 3000 samples
Epoch 1/1
27000/27000 [==============================] - 7s 272us/step - loss: 0.0202 - acc: 0.9978 - val_loss: 0.0803 - val_acc: 0.9719
Q   309-0 T -903 ☑ -903
Q 608-935 T -267 ☑ -267
Q  874-67 T -402 ☑ -402
Q  919-86 T -851 ☑ -851
Q 839-978 T -59  ☒ -69 
Q   4-648 T 842  ☑ 842 
Q  54-503 T 260  ☑ 260 
Q   959-7 T -952 ☑ -952
Q   496-6 T -688 ☑ -688
Q 062-319 T 653  ☑ 653 


# Testing

In [20]:
print("MSG : Prediction")
#####################################################
## Try to test and evaluate your model ##############
## ex. test_x = ["555+175", "860+7  ", "340+29 "]
## ex. test_y = ["730 ", "867 ", "369 "] 


preds = model.predict_classes(test_x, verbose=0)

i = 0
count = 0
while i < len(preds) :
    q = ctable.decode(test_x[i])
    correct = ctable.decode(test_y[i])
    guess = ctable.decode(preds[i], calc_argmax=False)

    print('Q', q, end=' ')
    print('T', correct, end=' ')
    if correct == guess:
        print(colors.ok + '☑' + colors.close, end=' ')
        count += 1
    else:
        print(colors.fail + '☒' + colors.close, end=' ')
    print(guess)
    i += 1
    
testing_acc = (count/len(test_y))*100
print('-'*60)
print('-'*30, colors.ok + str(testing_acc) + "%" + colors.close, '-'*30)
print('-'*60)
#####################################################
    

MSG : Prediction
Q 118-2   T 116  ☑ 116 
Q 68-49   T 19   ☑ 19  
Q 404-4   T 400  ☒ 300 
Q 803-354 T 449  ☒ 459 
Q 262-636 T -374 ☑ -374
Q 57-668  T -611 ☑ -611
Q 833-792 T 41   ☒ 31  
Q 97-7    T 90   ☒ 91  
Q 378-21  T 357  ☑ 357 
Q 3-731   T -728 ☑ -728
Q 757-664 T 93   ☒ 19  
Q 1-256   T -255 ☑ -255
Q 16-174  T -158 ☑ -158
Q 14-790  T -776 ☑ -776
Q 1-749   T -748 ☑ -748
Q 1-443   T -442 ☑ -442
Q 420-694 T -274 ☑ -274
Q 4-721   T -717 ☑ -717
Q 13-67   T -54  ☑ -54 
Q 26-27   T -1   ☑ -1  
Q 63-406  T -343 ☑ -343
Q 190-29  T 161  ☑ 161 
Q 413-34  T 379  ☑ 379 
Q 1-511   T -510 ☑ -510
Q 92-946  T -854 ☑ -854
Q 547-45  T 502  ☑ 502 
Q 577-63  T 514  ☑ 514 
Q 242-165 T 77   ☑ 77  
Q 56-470  T -414 ☑ -414
Q 153-29  T 124  ☑ 124 
Q 87-426  T -339 ☑ -339
Q 438-36  T 402  ☑ 402 
Q 37-310  T -273 ☑ -273
Q 404-67  T 337  ☑ 337 
Q 52-452  T -400 ☑ -400
Q 514-13  T 501  ☑ 501 
Q 9-339   T -330 ☑ -330
Q 224-519 T -295 ☑ -295
Q 39-33   T 6    ☑ 6   
Q 229-416 T -187 ☑ -187
Q 354-71  T 283  ☑ 283 

Q 235-7   T 228  ☑ 228 
Q 526-27  T 499  ☑ 499 
Q 18-523  T -505 ☑ -505
Q 971-330 T 641  ☑ 641 
Q 2-22    T -20  ☑ -20 
Q 45-488  T -443 ☑ -443
Q 48-538  T -490 ☑ -490
Q 2-813   T -811 ☑ -811
Q 38-38   T 0    ☑ 0   
Q 723-902 T -179 ☒ -189
Q 23-852  T -829 ☑ -829
Q 683-565 T 118  ☑ 118 
Q 99-73   T 26   ☑ 26  
Q 769-970 T -201 ☑ -201
Q 630-1   T 629  ☑ 629 
Q 949-934 T 15   ☒ 14  
Q 233-89  T 144  ☑ 144 
Q 73-405  T -332 ☑ -332
Q 974-358 T 616  ☑ 616 
Q 883-2   T 881  ☑ 881 
Q 897-48  T 849  ☑ 849 
Q 833-973 T -140 ☒ -130
Q 755-119 T 636  ☑ 636 
Q 9-967   T -958 ☑ -958
Q 425-745 T -320 ☑ -320
Q 53-570  T -517 ☑ -517
Q 120-406 T -286 ☑ -286
Q 830-5   T 825  ☑ 825 
Q 14-931  T -917 ☑ -917
Q 0-960   T -960 ☑ -960
Q 865-58  T 807  ☑ 807 
Q 840-281 T 559  ☑ 559 
Q 887-111 T 776  ☑ 776 
Q 588-9   T 579  ☑ 579 
Q 674-145 T 529  ☑ 529 
Q 86-148  T -62  ☑ -62 
Q 552-709 T -157 ☑ -157
Q 324-44  T 280  ☑ 280 
Q 9-215   T -206 ☑ -206
Q 50-763  T -713 ☑ -713
Q 396-6   T 390  ☑ 390 
Q 43-42   T 1   

Q 878-79  T 799  ☑ 799 
Q 609-378 T 231  ☑ 231 
Q 82-59   T 23   ☑ 23  
Q 8-194   T -186 ☑ -186
Q 47-450  T -403 ☑ -403
Q 77-140  T -63  ☑ -63 
Q 56-704  T -648 ☑ -648
Q 427-64  T 363  ☑ 363 
Q 57-39   T 18   ☑ 18  
Q 16-945  T -929 ☑ -929
Q 83-34   T 49   ☑ 49  
Q 638-392 T 246  ☑ 246 
Q 634-236 T 398  ☒ 498 
Q 57-27   T 30   ☑ 30  
Q 914-42  T 872  ☑ 872 
Q 7-204   T -197 ☑ -197
Q 36-278  T -242 ☑ -242
Q 989-142 T 847  ☑ 847 
Q 213-16  T 197  ☑ 197 
Q 35-712  T -677 ☑ -677
Q 36-90   T -54  ☑ -54 
Q 357-2   T 355  ☑ 355 
Q 291-84  T 207  ☑ 207 
Q 22-265  T -243 ☑ -243
Q 853-7   T 846  ☑ 846 
Q 870-551 T 319  ☑ 319 
Q 721-146 T 575  ☑ 575 
Q 11-347  T -336 ☑ -336
Q 452-810 T -358 ☑ -358
Q 726-778 T -52  ☑ -52 
Q 245-579 T -334 ☑ -334
Q 12-25   T -13  ☑ -13 
Q 11-348  T -337 ☑ -337
Q 746-9   T 737  ☑ 737 
Q 884-15  T 869  ☒ 879 
Q 164-755 T -591 ☑ -591
Q 130-829 T -699 ☒ -799
Q 454-11  T 443  ☑ 443 
Q 250-44  T 206  ☑ 206 
Q 645-401 T 244  ☑ 244 
Q 910-838 T 72   ☒ 73  
Q 335-29  T 306 

Q 26-365  T -339 ☑ -339
Q 43-769  T -726 ☑ -726
Q 878-166 T 712  ☑ 712 
Q 566-904 T -338 ☑ -338
Q 143-76  T 67   ☑ 67  
Q 340-56  T 284  ☑ 284 
Q 56-403  T -347 ☑ -347
Q 476-30  T 446  ☑ 446 
Q 62-470  T -408 ☑ -408
Q 887-12  T 875  ☑ 875 
Q 336-28  T 308  ☑ 308 
Q 72-49   T 23   ☑ 23  
Q 876-149 T 727  ☑ 727 
Q 666-9   T 657  ☑ 657 
Q 81-577  T -496 ☑ -496
Q 896-691 T 205  ☑ 205 
Q 650-44  T 606  ☑ 606 
Q 54-160  T -106 ☒ -105
Q 439-151 T 288  ☑ 288 
Q 632-936 T -304 ☑ -304
Q 12-55   T -43  ☑ -43 
Q 98-263  T -165 ☑ -165
Q 862-307 T 555  ☑ 555 
Q 781-91  T 690  ☑ 690 
Q 85-87   T -2   ☑ -2  
Q 200-187 T 13   ☒ 24  
Q 91-485  T -394 ☑ -394
Q 76-14   T 62   ☑ 62  
Q 510-18  T 492  ☑ 492 
Q 936-2   T 934  ☑ 934 
Q 857-95  T 762  ☑ 762 
Q 954-12  T 942  ☑ 942 
Q 842-5   T 837  ☑ 837 
Q 758-226 T 532  ☑ 532 
Q 589-44  T 545  ☑ 545 
Q 587-7   T 580  ☑ 580 
Q 602-622 T -20  ☒ -10 
Q 281-9   T 272  ☑ 272 
Q 985-792 T 193  ☒ 103 
Q 42-59   T -17  ☑ -17 
Q 7-162   T -155 ☑ -155
Q 420-136 T 284 

Q 65-501  T -436 ☑ -436
Q 64-448  T -384 ☑ -384
Q 86-530  T -444 ☑ -444
Q 494-73  T 421  ☑ 421 
Q 447-77  T 370  ☑ 370 
Q 384-95  T 289  ☒ 299 
Q 210-453 T -243 ☑ -243
Q 5-537   T -532 ☑ -532
Q 662-43  T 619  ☑ 619 
Q 919-78  T 841  ☑ 841 
Q 33-243  T -210 ☑ -210
Q 867-911 T -44  ☑ -44 
Q 895-50  T 845  ☑ 845 
Q 99-509  T -410 ☑ -410
Q 593-2   T 591  ☑ 591 
Q 30-470  T -440 ☑ -440
Q 219-290 T -71  ☒ -72 
Q 681-602 T 79   ☒ 78  
Q 4-996   T -992 ☑ -992
Q 270-240 T 30   ☑ 30  
Q 9-686   T -677 ☑ -677
Q 999-448 T 551  ☒ 541 
Q 193-26  T 167  ☑ 167 
Q 97-568  T -471 ☑ -471
Q 95-607  T -512 ☑ -512
Q 225-185 T 40   ☒ 41  
Q 861-945 T -84  ☒ -94 
Q 82-128  T -46  ☑ -46 
Q 380-720 T -340 ☑ -340
Q 685-6   T 679  ☑ 679 
Q 490-359 T 131  ☑ 131 
Q 2-339   T -337 ☑ -337
Q 2-172   T -170 ☑ -170
Q 84-193  T -109 ☑ -109
Q 482-67  T 415  ☑ 415 
Q 303-25  T 278  ☑ 278 
Q 790-3   T 787  ☑ 787 
Q 65-531  T -466 ☑ -466
Q 532-48  T 484  ☑ 484 
Q 92-994  T -902 ☑ -902
Q 77-160  T -83  ☑ -83 
Q 941-23  T 918 

...

Q 229-32  T 197  ☑ 197 
Q 2-215   T -213 ☑ -213
Q 248-56  T 192  ☑ 192 
Q 868-74  T 794  ☑ 794 
Q 636-9   T 627  ☑ 627 
Q 841-15  T 826  ☑ 826 
Q 327-599 T -272 ☒ -262
Q 32-316  T -284 ☑ -284
Q 32-165  T -133 ☑ -133
Q 91-300  T -209 ☒ -219
Q 48-739  T -691 ☑ -691
Q 9-471   T -462 ☑ -462
Q 248-154 T 94   ☒ 104 
Q 344-7   T 337  ☑ 337 
Q 593-888 T -295 ☑ -295
Q 99-937  T -838 ☑ -838
Q 998-60  T 938  ☑ 938 
Q 323-850 T -527 ☑ -527
Q 568-625 T -57  ☒ -47 
Q 875-513 T 362  ☑ 362 
Q 3-241   T -238 ☑ -238
Q 98-918  T -820 ☑ -820
Q 30-427  T -397 ☑ -397
Q 159-830 T -671 ☑ -671
Q 25-506  T -481 ☑ -481
Q 52-13   T 39   ☑ 39  
Q 83-68   T 15   ☑ 15  
Q 618-39  T 579  ☑ 579 
Q 806-239 T 567  ☒ 577 
Q 55-445  T -390 ☑ -390
Q 88-530  T -442 ☑ -442
Q 544-10  T 534  ☑ 534 
Q 269-93  T 176  ☑ 176 
Q 195-307 T -112 ☒ -122
Q 47-703  T -656 ☑ -656
Q 18-292  T -274 ☑ -274
Q 88-904  T -816 ☑ -816
Q 358-255 T 103  ☑ 103 
Q 91-66   T 25   ☑ 25  
Q 373-511 T -138 ☑ -138
Q 63-928  T -865 ☑ -865
Q 160-3   T 157 

Q 77-10   T 67   ☑ 67  
Q 509-862 T -353 ☑ -353
Q 63-323  T -260 ☑ -260
Q 697-64  T 633  ☑ 633 
Q 3-199   T -196 ☑ -196
Q 24-93   T -69  ☑ -69 
Q 59-728  T -669 ☑ -669
Q 169-80  T 89   ☒ 99  
Q 833-715 T 118  ☑ 118 
Q 587-532 T 55   ☑ 55  
Q 201-803 T -602 ☑ -602
Q 649-1   T 648  ☑ 648 
Q 317-738 T -421 ☑ -421
Q 875-493 T 382  ☑ 382 
Q 1-319   T -318 ☑ -318
Q 57-537  T -480 ☑ -480
Q 427-3   T 424  ☑ 424 
Q 293-2   T 291  ☑ 291 
Q 997-773 T 224  ☑ 224 
Q 986-93  T 893  ☑ 893 
Q 258-109 T 149  ☒ 159 
Q 27-68   T -41  ☑ -41 
Q 6-971   T -965 ☑ -965
Q 514-62  T 452  ☑ 452 
Q 31-917  T -886 ☑ -886
Q 510-11  T 499  ☒ 409 
Q 962-5   T 957  ☑ 957 
Q 293-971 T -678 ☒ -688
Q 696-37  T 659  ☑ 659 
Q 2-915   T -913 ☑ -913
Q 78-243  T -165 ☑ -165
Q 150-43  T 107  ☑ 107 
Q 119-58  T 61   ☑ 61  
Q 62-467  T -405 ☑ -405
Q 887-5   T 882  ☑ 882 
Q 521-57  T 464  ☑ 464 
Q 57-876  T -819 ☑ -819
Q 540-36  T 504  ☑ 504 
Q 372-54  T 318  ☑ 318 
Q 630-41  T 589  ☑ 589 
Q 871-52  T 819  ☑ 819 
Q 68-57   T 11  

Q 28-560  T -532 ☑ -532
Q 573-131 T 442  ☑ 442 
Q 61-5    T 56   ☑ 56  
Q 0-440   T -440 ☑ -440
Q 181-70  T 111  ☑ 111 
Q 24-357  T -333 ☑ -333
Q 40-465  T -425 ☑ -425
Q 14-702  T -688 ☑ -688
Q 923-991 T -68  ☒ -78 
Q 67-619  T -552 ☑ -552
Q 0-929   T -929 ☑ -929
Q 559-242 T 317  ☑ 317 
Q 590-99  T 491  ☑ 491 
Q 46-141  T -95  ☑ -95 
Q 335-60  T 275  ☑ 275 
Q 99-383  T -284 ☑ -284
Q 780-69  T 711  ☑ 711 
Q 690-81  T 609  ☑ 609 
Q 808-3   T 805  ☑ 805 
Q 189-75  T 114  ☑ 114 
Q 637-2   T 635  ☑ 635 
Q 264-519 T -255 ☑ -255
Q 224-971 T -747 ☑ -747
Q 990-22  T 968  ☑ 968 
Q 198-0   T 198  ☑ 198 
Q 26-719  T -693 ☑ -693
Q 193-827 T -634 ☑ -634
Q 70-648  T -578 ☑ -578
Q 891-97  T 794  ☑ 794 
Q 93-64   T 29   ☑ 29  
Q 85-100  T -15  ☑ -15 
Q 96-713  T -617 ☑ -617
Q 432-16  T 416  ☑ 416 
Q 4-953   T -949 ☑ -949
Q 375-19  T 356  ☑ 356 
Q 878-8   T 870  ☒ 860 
Q 821-62  T 759  ☑ 759 
Q 827-638 T 189  ☒ 199 
Q 575-849 T -274 ☑ -274
Q 47-27   T 20   ☑ 20  
Q 597-11  T 586  ☑ 586 
Q 5-105   T -100

Q 193-851 T -658 ☑ -658
Q 829-63  T 766  ☑ 766 
Q 45-167  T -122 ☑ -122
Q 51-719  T -668 ☑ -668
Q 96-919  T -823 ☑ -823
Q 84-286  T -202 ☑ -202
Q 31-905  T -874 ☑ -874
Q 521-87  T 434  ☑ 434 
Q 416-530 T -114 ☑ -114
Q 137-20  T 117  ☒ 107 
Q 441-56  T 385  ☑ 385 
Q 9-245   T -236 ☑ -236
Q 238-697 T -459 ☑ -459
Q 179-173 T 6    ☒ 1   
Q 403-135 T 268  ☑ 268 
Q 134-15  T 119  ☑ 119 
Q 0-116   T -116 ☑ -116
Q 28-272  T -244 ☑ -244
Q 79-634  T -555 ☑ -555
Q 7-212   T -205 ☑ -205
Q 125-97  T 28   ☑ 28  
Q 784-838 T -54  ☑ -54 
Q 918-91  T 827  ☑ 827 
Q 9-393   T -384 ☑ -384
Q 857-645 T 212  ☑ 212 
Q 970-38  T 932  ☑ 932 
Q 18-7    T 11   ☑ 11  
Q 113-10  T 103  ☑ 103 
Q 766-977 T -211 ☑ -211
Q 6-386   T -380 ☑ -380
Q 809-68  T 741  ☑ 741 
Q 275-487 T -212 ☑ -212
Q 2-573   T -571 ☑ -571
Q 46-306  T -260 ☑ -260
Q 328-55  T 273  ☑ 273 
Q 797-40  T 757  ☑ 757 
Q 8-283   T -275 ☑ -275
Q 329-714 T -385 ☑ -385
Q 468-574 T -106 ☑ -106
Q 423-6   T 417  ☑ 417 
Q 11-498  T -487 ☑ -487
Q 529-398 T 131 

Q 361-838 T -477 ☑ -477
Q 2-337   T -335 ☑ -335
Q 430-89  T 341  ☑ 341 
Q 611-404 T 207  ☒ 217 
Q 752-167 T 585  ☑ 585 
Q 93-458  T -365 ☑ -365
Q 295-10  T 285  ☑ 285 
Q 81-455  T -374 ☑ -374
Q 77-914  T -837 ☑ -837
Q 81-774  T -693 ☑ -693
Q 972-600 T 372  ☑ 372 
Q 582-51  T 531  ☑ 531 
Q 82-653  T -571 ☑ -571
Q 254-631 T -377 ☑ -377
Q 437-773 T -336 ☑ -336
Q 232-330 T -98  ☑ -98 
Q 446-68  T 378  ☑ 378 
Q 693-741 T -48  ☑ -48 
Q 840-84  T 756  ☑ 756 
Q 4-378   T -374 ☑ -374
Q 34-115  T -81  ☑ -81 
Q 313-44  T 269  ☑ 269 
Q 358-588 T -230 ☑ -230
Q 235-885 T -650 ☑ -650
Q 253-519 T -266 ☑ -266
Q 42-88   T -46  ☑ -46 
Q 420-568 T -148 ☑ -148
Q 29-92   T -63  ☑ -63 
Q 5-650   T -645 ☑ -645
Q 217-465 T -248 ☑ -248
Q 274-77  T 197  ☑ 197 
Q 61-26   T 35   ☑ 35  
Q 99-51   T 48   ☑ 48  
Q 23-333  T -310 ☑ -310
Q 604-49  T 555  ☑ 555 
Q 71-421  T -350 ☑ -350
Q 8-103   T -95  ☑ -95 
Q 722-613 T 109  ☑ 109 
Q 32-447  T -415 ☑ -415
Q 966-95  T 871  ☑ 871 
Q 626-562 T 64   ☒ 53  
Q 941-5   T 936 